# Erstmal "Run -> Run All Cells"

Vielleicht auch "View -> Collaps all code"

In [1]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

print("Loading modules (0/3)...")
with suppress_stdout():
    %pip install matplotlib
print("Module 1 loaded (1/3)...")
      
with suppress_stdout():
    %pip install ZODB
print("Module 2 loaded (2/3)...")
      
with suppress_stdout():
    %pip install PyGithub
    %matplotlib inline
print("Module 3 loaded (3/3)...Done!")
print()
    
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from DatabaseManagement import LeaderBoard, Player, Match, dbConnection, selectLeaderBoard
import TM_util as TM
database = dbConnection("TheRealShit.fs")
selected = selectLeaderBoard(database.root, "Terraforming Mars")

Loading modules (0/3)...
Module 1 loaded (1/3)...
Module 2 loaded (2/3)...
Module 3 loaded (3/3)...Done!

Connection established
You are now logged into Terraforming Mars's Leaderbord.


# Willkommen in der ELO Hell!

In [2]:
wallpaper = open("Wallpaper.jpg", "rb").read()
widgets.Image(value=wallpaper, format='jpg')

Image(value=b'\xff\xd8\xff\xe1\x14\x89Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x0e\x01\x00\x00\x03\x00\x00\x00\…

## Trage ein neues Spiel ein

Der Reihe nach die Plätze eintragen. Punkte werden dabei nur als Zusatzinfo betrachtet. Die richtige Reihenfolge ist also entscheidend

In [3]:
playerNames = [player.name for player in selected.sortedPlayerList(ordering="matches")]

matchDate  = widgets.DatePicker(description='Datum:')
Place, ScoreBox, ScoreSlide, ScoreLink, Score, Info, Box, Exp = [[],[],[],[],[],[],[],[]]

for i in range(5):
    Place.append(widgets.Combobox(placeholder="Wähle Spieler", options=playerNames, description = TM.nameOfPlaces[i+1]))
    ScoreBox.append(widgets.IntText(description = "Punkte:"))
    ScoreSlide.append(widgets.IntSlider(min=0, max=200, readout=False,  layout=widgets.Layout(width="70%")))
    ScoreLink.append(widgets.jslink((ScoreBox[i], 'value'), (ScoreSlide[i], 'value')))
    Score.append(widgets.VBox(children=[ScoreBox[i], ScoreSlide[i]], layout=widgets.Layout(align_items="flex-end")))
    Info.append(widgets.Combobox(placeholder="-", options=TM.listOfCorporations, description = "Konzern:"))
    Box.append(widgets.Box(children=[Place[i], Score[i], Info[i]]))

Exp.append(widgets.Checkbox(value=True, description='Prelude', layout=widgets.Layout(width='20%')))
Exp.append(widgets.Checkbox(value=True, description='Venus', layout=widgets.Layout(width='20%')))
Exp.append(widgets.Checkbox(value=True, description='Colonies', layout=widgets.Layout(width='20%')))
Exp.append(widgets.Checkbox(value=False, description='Turmoil', layout=widgets.Layout(width='20%')))
Exp.append(widgets.Dropdown(options=TM.listOfMaps, layout=widgets.Layout(width='20%')))
ExpBox = widgets.Box(children=Exp)

matchButton = widgets.Button(description="Spiel eintragen!")
matchButton2= widgets.Button(description="Weiteres Spiel eintragen", disabled=True, layout = widgets.Layout(width='auto'))
matchButtonBox = widgets.HBox(children=[matchButton, matchButton2])
matchOutput    = widgets.Output()

display(matchDate, Box[0], Box[1], Box[2] , Box[3], Box[4], ExpBox, matchButtonBox, matchOutput)

def on_matchButton_clicked(b):
    b.description="Submitted"
    b.disabled = True
    matchButton2.disabled = False
    b.icon="success"
    
    inputPlace, inputScore, inputAdditionalInfo, inputExpansions = [[],[],[],[]]
    for i in range(5):
        if Place[i].value == "":
            break
        else:
            inputPlace.append(Place[i].value)
            inputScore.append(str(ScoreBox[i].value))
            inputAdditionalInfo.append(Info[i].value)
    
    for i in range(4):
        if Exp[i].value == True:
            inputExpansions.append(Exp[i].description)
    
    inputExpansions.append(Exp[4].value)
    with matchOutput:
        selected.addMatch(str(matchDate.value), inputPlace, inputScore, inputAdditionalInfo, inputExpansions)
    
def on_matchButton2_clicked(b):
    b.disabled = True
    matchButton.disabled = False
    matchButton.description = "Spiel eintragen!"
    


matchButton.on_click(on_matchButton_clicked)
matchButton2.on_click(on_matchButton2_clicked)

DatePicker(value=None, description='Datum:')

Box(children=(Combobox(value='', description='Erster: ', options=('Lax', 'Felix', 'Simon', 'Nick', 'Steve', 'J…

Box(children=(Combobox(value='', description='Zweiter: ', options=('Lax', 'Felix', 'Simon', 'Nick', 'Steve', '…

Box(children=(Combobox(value='', description='Dritter: ', options=('Lax', 'Felix', 'Simon', 'Nick', 'Steve', '…

Box(children=(Combobox(value='', description='Vierter: ', options=('Lax', 'Felix', 'Simon', 'Nick', 'Steve', '…

Box(children=(Combobox(value='', description='Fünfter: ', options=('Lax', 'Felix', 'Simon', 'Nick', 'Steve', '…

Box(children=(Checkbox(value=True, description='Prelude', layout=Layout(width='20%')), Checkbox(value=True, de…

Output()

## Die Datenbank hochladen

Nicht vergessen sonst vergisst es.

In [4]:
uploadAccess  = widgets.Text(placeholder='Gif access', description='Access:')
uploadConfirm = widgets.Button(description='Datenbank hochladen')
uploadOutput  = widgets.Output()

display(uploadAccess, uploadConfirm, uploadOutput)

def on_uploadConfirm_clicked(b):
    b.disabled = True
    try:
        database.upload(uploadAccess.value)
        b.description="Erfolg!"
        b.icon="success"
    except:
        with uploadOutput:
            print("Etwas ist schief gelaufen")
        b.disabled = False

uploadConfirm.on_click(on_uploadConfirm_clicked)

Text(value='', description='Access:', placeholder='Gif access')

Button(description='Datenbank hochladen', style=ButtonStyle())

Output()

## Plotte die Entwicklung des ELOs der ausgewählten Spieler

Voreingestellt sind Spieler mit 5 oder mehr Spielen.

In [5]:
playerObjects = list(selected.sortedPlayerList(ordering="rank"))
nbrPlayers = len(playerObjects)
selPlayers = [widgets.ToggleButton(value=False, description="Normalize")]
for player in playerObjects:
    isEstablished = True if player.nbrOfMatches >= 5 else False
    selPlayers.append(widgets.ToggleButton(value=isEstablished, description=player.name, layout=widgets.Layout(height='4%')))
selBox   = widgets.VBox(children=selPlayers, layout=widgets.Layout(align_items="flex-start"))
graphOut = widgets.Output()
dispBox  = widgets.HBox(children=[selBox,graphOut])
display(dispBox)
with graphOut:
    playersToPlotList = [player.description for player in selPlayers if player.value==True]
    selected.plotSpecELO(playersToPlotList, normalized=False)

def on_value_change(change):
    with graphOut:
        graphOut.clear_output()
        playersToPlotList = [player.description for player in selPlayers[1:] if player.value==True]
        selected.plotSpecELO(playersToPlotList, normalized=selPlayers[0].value)
        
for selection in selPlayers:
    selection.observe(on_value_change, names='value')


## Lasse dir eine Liste von gemeinsamen Spielen anzeigen

Neueste Spiele zuerst.
Den gleichen Namen einzustellen zeigt alle Spiele eines Spielers

In [6]:
maxGamesPlayed = selected.playerList[playerNames[0]].nbrOfMatches
interact(selected.jointMatch,
         player1=widgets.Dropdown(options=playerNames, description='Spieler 1'),
         player2=widgets.Dropdown(options=playerNames, description='Spieler 2'),
         nbrOfMatches=widgets.IntSlider(min=1, max=maxGamesPlayed, value=5, description='Matches'));

interactive(children=(Dropdown(description='Spieler 1', options=('Lax', 'Felix', 'Simon', 'Nick', 'Steve', 'Ju…

## Lasse dir die Gewinnchancen vorhersagen

HEIZT DIE KONKURENZ AN!!!!

In [7]:
interact(selected.compareMatchup, player1=playerNames, player2=playerNames);

interactive(children=(Dropdown(description='player1', options=('Lax', 'Felix', 'Simon', 'Nick', 'Steve', 'Jule…

## Lege einen neuen Spieler an oder verwische deine Spuren

Es ist Vorsicht geboten! Vor allem beim Löschen von Spielen!!

In [8]:
newPlayerLabel   = widgets.Label('Neuer Spieler:', layout=widgets.Layout(width='130px'))
newPlayerText    = widgets.Text(placeholder='Alias')
newPlayerButton  = widgets.Button(description='Spieler anlegen')
newPlayerOutput  = widgets.Output()
newPlayerBox     = widgets.HBox(children=[newPlayerLabel, newPlayerText, newPlayerButton])

deleteGameLabel  = widgets.Label('Letztes Spiel löschen:', layout=widgets.Layout(width='130px'))
deleteGamePw     = widgets.Password(placeholder='Passwort')
deleteGameButton = widgets.Button(description='Spiel löschen')
deleteGameBox    = widgets.HBox(children=[deleteGameLabel, deleteGamePw, deleteGameButton])
deleteGameOutput = widgets.Output()

display(newPlayerBox, newPlayerOutput, deleteGameBox, deleteGameOutput)

def on_newPlaybeButton_clicked(b):
    with newPlayerOutput:
        selected.addPlayer(newPlayerText.value)

def on_deleteGameButton_clicked(b):
    if deleteGamePw.value=="ORDER66":
        deleteGamePw.value=""
        with deleteGameOutput:
            selected.deleteLastMatch()
    else:
        with deleteGameOutput:
            print("Falsches Passwort")

newPlayerButton.on_click(on_newPlaybeButton_clicked)
deleteGameButton.on_click(on_deleteGameButton_clicked)

Output()

Output()